In [ ]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/bus_terminals.csv')

In [2]:
venues = venues.dropna()

In [3]:
import folium

# Create a map centered at the average latitude and longitude
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Add markers for each cluster
for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m)


# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m)
    
# Add markers for each bus terminal
for _, row in bus_terminals.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='green')
    ).add_to(m)
# Display the map
m

/Users/leonardogreco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
demand= accomodations_clusters['total_accommodates'].to_numpy()
demand

array([ 6969,  6395,  2801,  7026,  4909,  1545,  6342,  6399,  1171,
        4117,  3110,  3088,  5080,  4779,  2666,  9095,  7555,  4679,
        2111,  3376,  7538,  7068,  6853,  1576,  2974,  2110,  1319,
       10383,  1268,   273,  8063,  8239,  3833,  6408,  7028,  2057,
        4099,  2060,  2646,  2505,  6751,   934,  4119,  9229,   563,
        7296,  2670,  2156,  2263,  1619, 12812])

In [7]:
from ortools.sat.python import cp_model

# Set the number of bus stops to locate
p = 15

# bus_matrix rows: accommodations; columns: candidate bus terminals
accom = list(bus_matrix.index)
bus_stops = list(bus_matrix.columns[1:])

model = cp_model.CpModel()

# Decision variables:
x = {j: model.NewBoolVar(f"x_{j}") for j in bus_stops}
y = {}
for idx, i in enumerate(accom):
    for j in bus_stops:
        y[(idx, j)] = model.NewBoolVar(f"y_{idx}_{j}")

# Constraint: each accommodation must be assigned to exactly one bus stop.
for idx, i in enumerate(accom):
    model.Add(sum(y[(idx, j)] for j in bus_stops) == 1)

# Constraint: assignment only possible if bus stop is selected.
for idx, i in enumerate(accom):
    for j in bus_stops:
        model.Add(y[(idx, j)] <= x[j])

# Constraint: exactly p bus stops are selected.
model.Add(sum(x[j] for j in bus_stops) == p)

# Objective: minimize total (demand-weighted) distance.
# We convert bus_matrix.loc[i, j] to an integer cost.
objective_terms = []
for idx, i in enumerate(accom):
    for j in bus_stops:
        # Multiply demand and the scaled distance.
        cost = int(round(bus_matrix.loc[i, j]))
        objective_terms.append(demand[idx] * cost * y[(idx, j)])
model.Minimize(sum(objective_terms))

# Solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    selected = [j for j in bus_stops if solver.Value(x[j]) == 1]
    print("Selected bus stops:", selected)
else:
    print("No solution found.")

Selected bus stops: ['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24', 'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']


In [8]:
#map only selected bus stops ['BD14', 'BD15', 'BT03', 'BL14', 'BT23', 'BT18', 'BL23', 'BL07', 'BT05']
# Add markers for each cluster
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m2 = folium.Map(location=map_center, zoom_start=10)

for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m2)
# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m2)

for _, row in bus_terminals[bus_terminals['id'].isin(selected)].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='lightgreen', icon='ok-sign')
    ).add_to(m2)
# Display the map with selected bus stops
m2

In [9]:
import math
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Build depot locations from selected bus terminals:
num_depots = len(selected)  # 15 depots, one for each vehicle
depots = []
for depot_id in selected:
    depot_row = bus_terminals[bus_terminals['id'] == depot_id].iloc[0]
    depots.append((depot_row['Latitude'], depot_row['Longitude']))

# Build regular nodes: first accommodation clusters, then venues.
regular_locations = []
regular_types = []  # record type as 'cluster' or 'venue'
for _, row in accomodations_clusters.iterrows():
    regular_locations.append((row['avg_latitude'], row['avg_longitude']))
    regular_types.append('cluster')
for _, row in venues.iterrows():
    regular_locations.append((row['Latitude'], row['Longitude']))
    regular_types.append('venue')

# Create full list of locations (depots come first)
locations = depots + regular_locations
num_locations = len(locations)

# Compute distance matrix (using Euclidean distance).
# When traveling from a regular node back to a depot, add a huge penalty if the node is a cluster,
# so that the solver will prefer ending a route on a venue.
def euclidean_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# Use a penalty value (tune as needed)
penalty = 10000

distance_matrix = [[0] * num_locations for _ in range(num_locations)]
for i in range(num_locations):
    for j in range(num_locations):
        # if going from a regular node (i >= num_depots) to a depot (j < num_depots), add penalty when node is a cluster.
        if i >= num_depots and j < num_depots:
            idx_regular = i - num_depots
            extra = penalty if regular_types[idx_regular] == 'cluster' else 0
            distance_matrix[i][j] = euclidean_distance(locations[i], locations[j]) + extra
        else:
            distance_matrix[i][j] = euclidean_distance(locations[i], locations[j])

# Create routing model:
num_vehicles = num_depots  # one vehicle per depot
# The first num_depots indices in locations are depots.
depot_indices = list(range(num_depots))

manager = pywrapcp.RoutingIndexManager(num_locations, num_vehicles, depot_indices, depot_indices)
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(distance_matrix[from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Set search parameters.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

# Solve the model.
solution = routing.SolveWithParameters(search_parameters)
if solution:
    for v in range(num_vehicles):
        index = routing.Start(v)
        route = []
        while True:
            next_index = solution.Value(routing.NextVar(index))
            if routing.IsEnd(next_index):
                break
            node = manager.IndexToNode(next_index)
            # We add only regular nodes (node indices that follow the depots)
            if node >= num_depots:
                route.append((node, regular_types[node - num_depots]))
            index = next_index
        print(f"Route for vehicle {v} (depot {selected[v]}): {route}")
else:
    print("No solution found.")

Route for vehicle 0 (depot BD14): []
Route for vehicle 1 (depot BD15): []
Route for vehicle 2 (depot BD16): []
Route for vehicle 3 (depot BT16): []
Route for vehicle 4 (depot BT03): []
Route for vehicle 5 (depot BT08): []
Route for vehicle 6 (depot BT24): []
Route for vehicle 7 (depot BL14): []
Route for vehicle 8 (depot BD04): []
Route for vehicle 9 (depot BT11): []
Route for vehicle 10 (depot BT06): []
Route for vehicle 11 (depot BT21): []
Route for vehicle 12 (depot BL20): []
Route for vehicle 13 (depot BT18): []
Route for vehicle 14 (depot BL23): [(97, 'venue'), (96, 'venue'), (95, 'venue'), (94, 'venue'), (93, 'venue'), (91, 'venue'), (92, 'venue'), (90, 'venue'), (89, 'venue'), (88, 'venue'), (86, 'venue'), (85, 'venue'), (84, 'venue'), (83, 'venue'), (82, 'venue'), (81, 'venue'), (80, 'venue'), (79, 'venue'), (78, 'venue'), (77, 'venue'), (76, 'venue'), (75, 'venue'), (74, 'venue'), (73, 'venue'), (72, 'venue'), (71, 'venue'), (70, 'venue'), (69, 'venue'), (68, 'venue'), (67, 'v